In [2]:
from concrete import fhe
import numpy as np

In [5]:
coord_x_rpeaks_no_107 = np.array(np.load('../Pan_Tompkins_QRS_Detection/extracted_data/coord_x_r_peaks_no_107.npy'))

# Sampling frequency (Hz)
fs = 360

array_len = len(coord_x_rpeaks_no_107)

@fhe.compiler({"x": "encrypted"})
def f(x):
    for i in range(array_len-1):
        x[i] = x[i+1]-x[i]

    peaks_mean = np.floor_divide(sum(x[:-1]) , (array_len-1))

    real_fs = np.multiply(60,fs)

    heart_rate = np.floor_divide(real_fs, peaks_mean)
    
    return heart_rate

inputset = [np.random.randint(4000, size=15), np.random.randint(4000, size=15),np.random.randint(4000, size=15)]
circuit = f.compile(inputset)

# Save everyting the server needs 
circuit.server.save("server.zip")

In [6]:
server = fhe.Server.load("server.zip")

# Need to wait for requests from clients. The first likely request is for ClientSpecs.
# Clients need ClientSpecs to generate keys and request computation
serialized_client_specs: str = server.client_specs.serialize()

# After getting the serialized ClientSpecs from a server, you can create the client object:
client_specs = fhe.ClientSpecs.deserialize(serialized_client_specs)
client = fhe.Client(client_specs)

# Keys generation
# seed of 0 will result in a crypto secure randomly generated 128-bit seed (see keys.py)
client.keys.generate(seed=0)

# server requires evaluation keys linked to the encryption keys, so we serialize them
serialized_evaluation_keys: bytes = client.evaluation_keys.serialize()

# encrypt your inputs and request the server to perform the computation
serialized_args: bytes = client.encrypt(coord_x_rpeaks_no_107).serialize()

deserialized_evaluation_keys = fhe.EvaluationKeys.deserialize(serialized_evaluation_keys)
deserialized_args  = server.client_specs.deserialize_public_args(serialized_args)

# perform computation
public_result = server.run(deserialized_args, deserialized_evaluation_keys)
serialized_public_result: bytes = public_result.serialize()

deserialized_public_result = client.specs.deserialize_public_result(serialized_public_result)

result = client.decrypt(deserialized_public_result)

print(result)

81
